In [1]:
import os
import sys
import random
import time
import select
from pathlib import Path

import h5py
import numpy as np
from tqdm import tqdm


import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.nn import BCELoss
from monai.losses.dice import GeneralizedDiceLoss
from torch.utils.data import DataLoader, TensorDataset
from torch.utils.data.dataset import Dataset

import socket
from loguru import logger

from model import dict_args, init_weights, FCT_Tail

In [2]:
device = torch.device("cpu")

In [3]:
loss_fn = BCELoss()

dic_loss_fn = GeneralizedDiceLoss(to_onehot_y=True, softmax=True)

In [4]:
# =======================================================================
#                                TAIL
# =======================================================================

model_tail = FCT_Tail()
model_tail.apply(init_weights)

optimizer_tail = torch.optim.AdamW(model_tail.parameters(), lr=dict_args['lr'],weight_decay=dict_args['decay'])

scheduler_tail = torch.optim.lr_scheduler.ReduceLROnPlateau(
            optimizer_tail,
            mode='min',
            factor=dict_args['lr_factor'],
            verbose=True,
            threshold=1e-6,
            patience=10,
            min_lr=dict_args['min_lr'])

model_tail.to(device)

print("Initialized ....")

Initialized ....


In [5]:
def send_socket_info(port=6161, message_data=None):
    import socket
    with socket.socket(socket.AF_INET, socket.SOCK_STREAM) as s:
        s.connect(('127.0.0.1', port))
        if message_data is  None:
            raise ValueError(f"Message that is being sent to port {port} is empty")
        s.send(message_data.encode('UTF-8'))

In [6]:
recv_head_y = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
recv_head_y.bind(('127.0.0.1', 6161))


recv_body_f = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
recv_body_f.bind(('127.0.0.1', 7171))

In [7]:
# Forward propagation in Tail model

model_tail.train()


model_tail.train()
train_loss_list = []
grads_dict = {}
abs_grads_dict = {}


try:
    while True:
        
        # wait until either socket is ready to read
        recv_head_y.listen()
        recv_body_f.listen()

        sockets = [recv_head_y, recv_body_f]

        read_sockets = []
        sockets_ready = 0
        
        logger.info(f" The read socket has total ready sockets of {len(read_sockets)}")

        label_data = None
        layer_data = None
        try:
            while (label_data is None and layer_data is None):
                logger.debug(f"starting at position {sockets_ready+1}")
                

                read_socket_head, _, _ = select.select([recv_head_y], [], [])

                if read_socket_head:
                    logger.debug("Socket Y label is ready")
                    conn, addr = recv_head_y.accept()
                    logger.info(f"connected tail to get data on label from head 6666: {addr}")

                    label_data_file = conn.recv(1024)

                    if len(label_data_file) == 0:
                        logger.error("Label data not available")
                        break

                    with h5py.File(label_data_file, 'r') as head_label:
                        label_data = torch.tensor(head_label['head_to_tail'][:]).to(device)
                    
                    logger.info("Fetched the label data from head.")

                time.sleep(8)
                read_socket_body, _, _ = select.select([recv_body_f], [], [])

                if read_socket_body:
                    logger.debug("Socket Body data is ready")
                    conn, addr = recv_body_f.accept()
                    logger.info(f"connected tail to get data on layer data from body 7171: {addr}")

                    body_data_file = conn.recv(1024)

                    if len(body_data_file) == 0:
                        logger.error("Layer data not available")
                        break

                    with h5py.File(body_data_file, 'r') as body_fwd:
                        layer_data = torch.tensor(body_fwd['skip9'][:], requires_grad=True).to(device)
                    
                    logger.info("Fetched the label data from head.")
                
                sockets_ready += 1
                
            
                
            optimizer_tail.zero_grad()
                                                                                                                                    
            tl_output_data = model_tail(# skip_7, skip_8, 
                layer_data
                )
            
            loss = loss_fn(tl_output_data, label_data)
            train_loss_list.append(loss)
            loss.backward()
            optimizer_tail.step()

            skip_9_grad = layer_data.grad

            with h5py.File('params_and_grads/tail_back_prop.hdf5', 'w') as tail_bkp:
                tail_bkp.create_dataset("skip_9_grad",  data=skip_9_grad.cpu().detach().numpy())

                send_socket_info(8181, 'params_and_grads/tail_back_prop.hdf5')

            logger.info(f"sent the backprop data from tail to body 8181 {skip_9_grad.shape}")
        except ConnectionResetError:
            logger.error(f"Connection error issue")

except Exception as ex:
    import traceback
    print("+=" * 25)
    print("Error encountered as :", ex)
    print("+=" * 25)
    traceback.print_exc()

finally:
    pass


2023-04-23 18:10:44.257 | INFO     | __main__:<module>:24 -  The read socket has total ready sockets of 0
2023-04-23 18:10:44.259 | DEBUG    | __main__:<module>:30 - starting at position 1
2023-04-23 18:10:53.310 | DEBUG    | __main__:<module>:36 - Socket Y label is ready
2023-04-23 18:10:53.313 | INFO     | __main__:<module>:38 - connected tail to get data on label from head 6666: ('127.0.0.1', 65519)
2023-04-23 18:10:53.321 | INFO     | __main__:<module>:49 - Fetched the label data from head.
2023-04-23 18:11:01.328 | DEBUG    | __main__:<module>:55 - Socket Body data is ready
2023-04-23 18:11:01.331 | INFO     | __main__:<module>:57 - connected tail to get data on layer data from body 7171: ('127.0.0.1', 65524)
2023-04-23 18:11:01.336 | INFO     | __main__:<module>:68 - Fetched the label data from head.
2023-04-23 18:11:01.462 | INFO     | __main__:<module>:92 - sent the backprop data from tail to body 8181 torch.Size([4, 8, 112, 112])
2023-04-23 18:11:01.464 | INFO     | __main__:<

DS 10 out -> [4, 1, 224, 224]


2023-04-23 18:11:24.295 | DEBUG    | __main__:<module>:36 - Socket Y label is ready
2023-04-23 18:11:24.303 | INFO     | __main__:<module>:38 - connected tail to get data on label from head 6666: ('127.0.0.1', 49152)
2023-04-23 18:11:24.312 | INFO     | __main__:<module>:49 - Fetched the label data from head.
2023-04-23 18:11:32.324 | DEBUG    | __main__:<module>:55 - Socket Body data is ready
2023-04-23 18:11:32.326 | INFO     | __main__:<module>:57 - connected tail to get data on layer data from body 7171: ('127.0.0.1', 49160)
2023-04-23 18:11:32.330 | INFO     | __main__:<module>:68 - Fetched the label data from head.
2023-04-23 18:11:32.421 | INFO     | __main__:<module>:92 - sent the backprop data from tail to body 8181 torch.Size([4, 8, 112, 112])
2023-04-23 18:11:32.422 | INFO     | __main__:<module>:24 -  The read socket has total ready sockets of 0
2023-04-23 18:11:32.423 | DEBUG    | __main__:<module>:30 - starting at position 1


DS 10 out -> [4, 1, 224, 224]


2023-04-23 18:11:48.117 | DEBUG    | __main__:<module>:36 - Socket Y label is ready
2023-04-23 18:11:48.120 | INFO     | __main__:<module>:38 - connected tail to get data on label from head 6666: ('127.0.0.1', 49172)
2023-04-23 18:11:48.125 | INFO     | __main__:<module>:49 - Fetched the label data from head.
2023-04-23 18:11:56.132 | DEBUG    | __main__:<module>:55 - Socket Body data is ready
2023-04-23 18:11:56.135 | INFO     | __main__:<module>:57 - connected tail to get data on layer data from body 7171: ('127.0.0.1', 49182)
2023-04-23 18:11:56.140 | INFO     | __main__:<module>:68 - Fetched the label data from head.
2023-04-23 18:11:56.214 | INFO     | __main__:<module>:92 - sent the backprop data from tail to body 8181 torch.Size([4, 8, 112, 112])
2023-04-23 18:11:56.215 | INFO     | __main__:<module>:24 -  The read socket has total ready sockets of 0
2023-04-23 18:11:56.217 | DEBUG    | __main__:<module>:30 - starting at position 1


...

DS 10 out -> [4, 1, 224, 224]


2023-04-23 18:19:12.928 | DEBUG    | __main__:<module>:36 - Socket Y label is ready
2023-04-23 18:19:12.930 | INFO     | __main__:<module>:38 - connected tail to get data on label from head 6666: ('127.0.0.1', 49987)
2023-04-23 18:19:12.934 | INFO     | __main__:<module>:49 - Fetched the label data from head.
2023-04-23 18:19:20.937 | DEBUG    | __main__:<module>:55 - Socket Body data is ready
2023-04-23 18:19:20.938 | INFO     | __main__:<module>:57 - connected tail to get data on layer data from body 7171: ('127.0.0.1', 49995)
2023-04-23 18:19:20.938 | INFO     | __main__:<module>:68 - Fetched the label data from head.
2023-04-23 18:19:21.022 | INFO     | __main__:<module>:92 - sent the backprop data from tail to body 8181 torch.Size([4, 8, 112, 112])
2023-04-23 18:19:21.022 | INFO     | __main__:<module>:24 -  The read socket has total ready sockets of 0
2023-04-23 18:19:21.022 | DEBUG    | __main__:<module>:30 - starting at position 1


DS 10 out -> [4, 1, 224, 224]


2023-04-23 18:19:34.378 | DEBUG    | __main__:<module>:36 - Socket Y label is ready
2023-04-23 18:19:34.379 | INFO     | __main__:<module>:38 - connected tail to get data on label from head 6666: ('127.0.0.1', 50179)
2023-04-23 18:19:34.382 | INFO     | __main__:<module>:49 - Fetched the label data from head.
2023-04-23 18:19:42.392 | DEBUG    | __main__:<module>:55 - Socket Body data is ready
2023-04-23 18:19:42.392 | INFO     | __main__:<module>:57 - connected tail to get data on layer data from body 7171: ('127.0.0.1', 50183)
2023-04-23 18:19:42.400 | INFO     | __main__:<module>:68 - Fetched the label data from head.
2023-04-23 18:19:42.472 | INFO     | __main__:<module>:92 - sent the backprop data from tail to body 8181 torch.Size([4, 8, 112, 112])
2023-04-23 18:19:42.475 | INFO     | __main__:<module>:24 -  The read socket has total ready sockets of 0
2023-04-23 18:19:42.475 | DEBUG    | __main__:<module>:30 - starting at position 1


DS 10 out -> [4, 1, 224, 224]
